In [ ]:
from pyjstat import pyjstat
import requests
import pandas as pd
import json
import ipywidgets as widgets

### 08531: Medlemmer i trus- og livssynssamfunn som mottek offentleg stønad og er utanfor Den norske kyrkja, etter religion/livssyn (F)
https://www.ssb.no/statbank/table/08531/

Dette notebooket henter data fra statbanken, kobler mot korrespondansetabell fra KLASS, for å kompansere for fylkes-sammenslåinger i 2017+2019.\
Arket resulterer i en interaktiv graf ved bruk av ipywidgets + matplotlib (innebygd i pandas)

### Hent data fra statbanken

In [ ]:
URL = 'http://data.ssb.no/api/v0/no/table/08531'

In [ ]:
payload = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:Fylker",
        "values": [
          "30",
          "01",
          "02",
          "06",
          "03",
          "34",
          "04",
          "05",
          "38",
          "07",
          "08",
          "42",
          "09",
          "10",
          "11",
          "46",
          "12",
          "13",
          "14",
          "15",
          "50",
          "16",
          "17",
          "18",
          "54",
          "19",
          "20",
          "21",
          "22",
          "23",
          "25",
          "26",
          "88",
          "99"
        ]
      }
    },
    {
      "code": "ReligionLivs",
      "selection": {
        "filter": "item",
        "values": [
          "999",
          "200",
          "400",
          "600",
          "902",
          "900"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}

In [ ]:
# Hent data fra statistikkbanken
resultat = requests.post(URL, json = payload)
# Les resultatet som ett pyjstat-class-objekt
dataset = pyjstat.Dataset.read(resultat.text)
# Skriv "hovedresultat" til en dataframe
df = dataset.write('dataframe')
df

### Fest på fylkeskode på hovedresultat

In [ ]:
# Hent og sett regionkoder inn i en dataframe
regionkode_dict = dict(dataset.get('dimension').get('Region').get('category').get('label'))
df_regionkode = pd.DataFrame(regionkode_dict.items(), columns=['fylkekode', 'region'])
 
# Koble koder på regioner i hoveddataframe utifra regionnavn
df = df.merge(df_regionkode, on = 'region' , how = 'left')
df

### Litt opprydding

In [ ]:
# Dropp kolonnen som er lik på hver linje
df.drop('statistikkvariabel', axis = "columns", inplace = True)
# Kutt ut oppsummeringsvariabelen
df = df[~df['religion/livssyn'].isin(['Medlemmer i alt'])]
# Døp om kolonner
df.columns = ['fylke', 'religion', 'aar', 'value', 'fylkekode']
# Konverter value (antall medlemmer) til int
df['value'] = df['value'].fillna(0)
df['value'] = df['value'].astype(int)
df

### Importer korrespondansetabell 2019

In [ ]:
# Hent korrespondansetabell
URL = "http://data.ssb.no/api/klass/v1/correspondencetables/452"
r = requests.get(URL)
# Konverter via json til en pandas dataframe av versjonene i klassifikasjonen
df_corr = r.content.decode('utf8').replace("'", '"')
df_corr = json.loads(df_corr)
corr_maps = df_corr['correspondenceMaps']

### Importer korrespondansetabell 2018/2017 (Trøndelag)

In [ ]:
# Hent korrespondansetabell
URL = "http://data.ssb.no/api/klass/v1/correspondencetables/588"
r = requests.get(URL)
# Konverter via json til en pandas dataframe av versjonene i klassifikasjonen
df_corr_tron = r.content.decode('utf8').replace("'", '"')
df_corr_tron = json.loads(df_corr_tron)
corr_maps_tron = df_corr_tron['correspondenceMaps']

# Kombiner med eksisterende korrespondansetabell
corr_maps += corr_maps_tron
corr_maps

### Kombiner tall fra de kombinerte fylkene ift. korrespondansetabellene

In [ ]:
# Funksjon for å finne en korrespondanse
def get_corr(code, src, tar):
    output = []
    for item in corr_maps:
        if item[src] == code:
            output.append(item[tar])
    # Vi går innom ett "set" for å fjerne duplikater i returnen, 
    # f.eks. når vi finner koden til "Viken" trenger vi ikke få "30" tre ganger
    return list(set(output))

In [ ]:
# Finn de unike source-idene (sammenslåtte fylker)
sammenslaatt_fylk = []
for item in corr_maps:
    sammenslaatt_fylk.append(item['sourceCode'])
sammenslaatt_fylk = list(set(sammenslaatt_fylk))

# Lag ett dictionary for koblingen mellom fylkene
fylk_kobl = {}
for fylk in sammenslaatt_fylk:
    fylk_kobl[fylk] = get_corr(fylk, 'sourceCode', 'targetCode')
    
fylk_kobl

In [ ]:
print('Fylkene som ligger i gjeldende fylkesinndeling')
r = requests.get('http://data.ssb.no/api/klass/v1/versions/1158')
gjeldende = r.text
gjeldende = r.content.decode('utf8').replace("'", '"')
gjeldende = json.loads(gjeldende)
gjeldende = pd.json_normalize(gjeldende['classificationItems'])

gjeld_fylk = list(gjeldende['name'])
gjeld_fylk

In [ ]:
# Lag en liste av alle fylkeskodene som skal kobles inn i andre
kobleliste = [item for sublist in fylk_kobl.values() for item in sublist]
kobler = []
mangler = []

# For hvert fylke, sjekk om den ligger i den gjeldende fylkesinndelingen
for fylk_kode in pd.unique(df[~df['fylkekode'].isin(list(gjeldende['code']))]['fylkekode']):
    # For de som ikke ligger der, sjekk om de skal kobles som ett "barn"
    if fylk_kode in kobleliste:
        kobler.append(fylk_kode)
    else: 
        # Disse er da hverken ett gjeldende fylke, eller noe som vil kobles inn i de gjeldende fylkene
        mangler.append(fylk_kode)    

# Om mangler-listen ikke er tom
if mangler:
    print('Fylkene i trossamfil som ikke er i gjeldende fylkesinndeling og ikke vil kobles\n')
    for code in mangler:
        #print(code)
        print(df_regionkode[df_regionkode['fylkekode'] == code]['region'].values[0])
    print('\n')

# Om kobler-listen ikke er tom
if kobler:
    print('Fylkene i trossamfil som ikke er i gjeldende fylkesinndeling, men som vil kobles\n')
    for code in kobler:
        print(df_regionkode[df_regionkode['fylkekode'] == code]['region'].values[0])
        

In [ ]:
# Koden i cellen under gir masse warnings, men fungerer for det, vi skrur derfor av warnings her frem til dette er gjennomgått
pd.set_option('mode.chained_assignment', None)

In [ ]:
# Loop gjennom dictionariet, og om summen er null på linjen, legg inn fra targets/kildefylker
for source, targets in fylk_kobl.items():
    # Finner linjer som har fylket vi ser på
    interest = df[df['fylkekode'] == source]
    # Og hvor verdien er null
    interest = interest[interest['value'] == 0]
    
    # Gå gjennom hver rad av dette
    for i, row in interest.iterrows():
        value_new = 0
        # Gå gjennom hver av de koblede fylkene og legg sammen verdien for året og religionen
        for target in targets:
           

            # Legg verdien fra kildefylket til det korresponderende fylket
            select = df[
                    (df['aar'] == row['aar']) & 
                    (df['religion'] == row['religion']) & 
                    (df['fylkekode'] == target) &
                    (df['value'] > 0)]
            if len(select) == 1:
                value_new += float(select['value'])
            else:
                print('\nSELECTION')
                print(select)
                print('*'*85)
                print('\nKILDERAD')
                print(row)
                print('*'*85)
                print('\nDATASETT PÅ MATCH')
                print(target)
                print(df[df['fylkekode'] == target])
                raise ValueError('Denne selectionen er ikke 1-rad lang, eller ett annet issue')
                
        df.loc[(df['fylkekode'] == source) & (df['religion'] == row['religion']) & (df['aar'] == row['aar']), 'value'] = value_new
        
df

In [ ]:
# Så skrur vi på warnings igjen
pd.set_option('mode.chained_assignment', 'warn')

In [ ]:
# Ekstra sjekk på hva som skjer med Trøndelag i overgangen 2015-2019
df_tron = df[df['fylkekode'] == "50"]
df_tron = df_tron.pivot('aar', 'religion', 'value')
df_tron.plot.area(figsize = (15,10), title = 'Trøndelag')

In [ ]:
# Ekstra sjekk på hva som skjer med Troms-Finnmark i overgangen 2015-2019
df_finn = df[df['fylkekode'] == '54']
df_finn = df_finn.pivot('aar', 'religion', 'value')
df_finn.plot.area(figsize = (15,10), title = 'Troms og Finnmark')

## Behold bare fylkene som finnes i fylkesinndelingen

In [ ]:
df = df[df['fylkekode'].isin(gjeldende['code'])]

In [ ]:
# Sjekk om det er noen fylker som mangler ift. fylkesinndelingen
for fylk in gjeldende['code']:
    if fylk not in list(pd.unique(df['fylkekode'])):
        print(f'{fylk} - Finnes i fylkesinndelingen, men ligger ikke i datasettet')

# Interaktiv graf med ipywidgets

In [ ]:
# Sorteringsrekkefølge for religioner etter størrelser på landsbasis (alle fylker) i siste året i datasettet
df_thisyr = df[df['aar'] == df['aar'].max()]
df_thisyr = df_thisyr.groupby('religion').sum().reset_index()
df_thisyr = df_thisyr.sort_values('value', ascending = False).reset_index()
df_thisyr

In [ ]:
# Fylkesdropdown
fylker = list(pd.unique(df['fylke']))
fylker.sort()
default = '-Velg Fylke-'
fylker.insert(0, default)

drop = widgets.Dropdown(
    options= fylker,
    value=fylker[0],
    description='Fylke:',
    disabled=False,
)

# Religion dropdown
relg = list(pd.unique(df['religion']))
relg.sort()
relg_default = '-Velg Organisasjon-'
relg.insert(0, relg_default)

drop_relg = widgets.Dropdown(
    options= relg,
    value=relg[0],
    description='Organisasjon:',
    disabled=False,
)

# Årsslider
min_aar = df['aar'].min()
max_aar = df['aar'].max()

rang = widgets.IntRangeSlider(
    value=[min_aar, max_aar],
    min= min_aar ,
    max= max_aar,
    step=1,
    description='År:',
    disabled=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)


# Update handler
def f(drop, relg, rang):    
    
    
    # Ta ny kopi av datasett
    df_tmp = df
    
    # Filtrer fylker
    if drop != default:
        df_tmp = df_tmp[df_tmp['fylke'] == drop]       
    
    # Filtrer organisasjon
    if relg != relg_default:
        df_tmp = df_tmp[df_tmp['religion'] == relg]
    
    # Filtrer år
    if str(rang[0]) != min_aar or str(rang[1]) != max_aar:
        df_tmp = df_tmp[(df_tmp['aar'].astype(int) >= int(rang[0])) & (df_tmp['aar'].astype(int) <= int(rang[1]))]
     
    # Grafen blir for komplisert om ingen valg er valgt, 
    # så vi forutsetter at man velger fylke eller organisasjon for å få se noe
    
    if drop != default or relg != relg_default:
        # Om man har valgt fylke, så bør religion være grupperingen
        if drop != default and relg == relg_default:
            print('Bruker religion som gruppering')
            df_tmp_piv = df_tmp.pivot('aar', 'religion', 'value')
            #print(df_tmp_piv)
            df_tmp_piv = df_tmp_piv[list(df_thisyr['religion'])]
        # Om man ikke har valgt organisasjon, så bør organisasjonene være grupperingene
        elif relg != relg_default and drop == default:
            print('Bruker fylke som gruppering')
            df_tmp_piv = df_tmp.pivot('aar', 'fylke', 'value')
        # Ellers har man valgt en av hver, og da er ikke grupperinger nødvendig
        else:
            print('Det vil bare være en gruppering')
            df_tmp_piv = df_tmp.pivot('aar', 'fylke', 'value')
    
        # Graf output
        df_tmp_piv.plot.area(figsize = (15,10))
        #print(df_tmp)
            
    else:
        print('Velg fylke eller organisasjon for å vise graf.')
    

# Koble output
out = widgets.interactive_output(f, {'drop': drop, 'relg': drop_relg, 'rang': rang})

# Launch layout
widgets.VBox([widgets.HBox([drop, drop_relg, rang]), out])